In [1]:
import ee
import geemap
import pandas as pd
import folium
from ipywidgets import Dropdown, Output, VBox

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()


In [18]:

# Define the ROIs (already provided)
roi_LF = ee.Geometry.Polygon(
    [[163.05418845709113, -77.62217141675877], 
    [163.04800864752082, -77.61908009873933], 
    [163.0795943408802, -77.61422077772508], 
    [163.08302756841925, -77.61569349746868],
    [163.09813376959113, -77.60950691594756], 
    [163.1407057910755, -77.60567560295046], 
    [163.15924521978644, -77.60228539245952], 
    [163.1736647754505, -77.60140083961204], 
    [163.17435142095832, -77.5996315475443],
    [163.18259116705207, -77.59786200695194],
    [163.19769736822394, -77.60125340809783], 
    [163.21417686041144, -77.60184312380088], 
    [163.2505690723255, -77.59874680831578], 
    [163.25812217291144, -77.60036878277327],
    [163.25400229986457, -77.60302247238047],
    [163.25468894537238, -77.60493867830638], 
    [163.25880881841925, -77.60670722499943], 
    [163.2334029346302, -77.60788611813062], 
    [163.21967002447394, -77.60655985559598],
    [163.20799705084113, -77.60980158403946],
    [163.21280356939582, -77.61156944773495], 
    [163.19357749517707, -77.61716604483516], 
    [163.18121787603644, -77.6155462332516], 
    [163.15237876470832, -77.61687154915111],
    [163.14894553716925, -77.62025783330262],
    [163.12010642584113, -77.62231860342517], 
    [163.10637351568488, -77.62217141675877], 
    [163.0795943408802, -77.61893287416414], 
    [163.06448813970832, -77.61849119009841],
    [163.06723472173957, -77.62099386140082],
    [163.06448813970832, -77.6226129715892],
    [163.05418845709113, -77.62217141675877]]
)

#section of clow island that near almost never has snow (eyeballed)
roi_soil_LF = ee.Geometry.Polygon([
    [163.1714023037232,-77.61219987779917], 
    [163.17492136195074,-77.61208939912747], 
    [163.18097242548833,-77.61243003858347], 
    [163.18303236201177,-77.61154619821772], 
    [163.17809709742437,-77.61084644723785], 
    [163.17290434077154,-77.61131602126838], 
    [163.1714023037232,-77.61219987779917]
])

#
roi_ice_LF = ee.Geometry.Polygon([
    [163.12882887006907,-77.60801384685202], 
    [163.12951551557688,-77.60915578272046], 
    [163.14453588606028,-77.6069823318652], 
    [163.13964353681712,-77.60668759777366],
    [163.12882887006907,-77.60801384685202]
])

roi_LF_crop = ee.Geometry.Polygon([[163.10147025602208,-77.63763858931995],
                                   [163.31227042692052,-77.61056024754215],
                                   [163.23605277555333,-77.5841617068026],
                                   [163.02731254117833,-77.6108548909756],
                                   [163.10147025602208,-77.63763858931995]
])

pointed = ee.Geometry.BBox(162.1, -77.73, 163.3, -77.59)

In [13]:
ids = ee.List(["LANDSAT_8_2016-11-02", "LANDSAT_8_2016-11-04", "LANDSAT_8_2016-11-06", "LANDSAT_8_2016-11-08",
               "LANDSAT_8_2016-11-13", "LANDSAT_8_2016-11-15", #"LANDSAT_8_2016-11-22", "LANDSAT_8_2016-11-24", # removed for heavy snow cover
               #"LANDSAT_8_2016-11-27",  # removed for heavy snow cover
               "LANDSAT_8_2016-12-08", "LANDSAT_8_2016-12-10", "LANDSAT_8_2016-12-13",
               "LANDSAT_8_2016-12-15", "LANDSAT_8_2016-12-17", "LANDSAT_8_2016-12-19", "LANDSAT_8_2016-12-24",
               "LANDSAT_8_2017-01-02", "LANDSAT_8_2017-01-09", "LANDSAT_8_2017-01-11", "LANDSAT_8_2017-01-14",
               "LANDSAT_8_2017-01-18", "LANDSAT_8_2017-01-25", "LANDSAT_8_2017-01-27", "LANDSAT_8_2017-01-30",
               "LANDSAT_8_2017-11-04", "LANDSAT_8_2017-11-07", "LANDSAT_8_2017-11-18", "LANDSAT_8_2017-11-20",
               "LANDSAT_8_2017-11-21", "LANDSAT_8_2017-11-25", "LANDSAT_8_2017-11-27", "LANDSAT_8_2017-11-30",
               "LANDSAT_8_2017-12-02", "LANDSAT_8_2017-12-07", "LANDSAT_8_2017-12-16", "LANDSAT_8_2017-12-23",
               "LANDSAT_8_2018-01-03", "LANDSAT_8_2018-01-05", "LANDSAT_8_2018-01-07", "LANDSAT_8_2018-01-10",
               "LANDSAT_8_2018-01-12", "LANDSAT_8_2018-01-14", "LANDSAT_8_2018-01-19", "LANDSAT_8_2018-01-26",
               "LANDSAT_8_2018-01-30", "LANDSAT_8_2018-11-05", "LANDSAT_8_2018-11-07", "LANDSAT_8_2018-11-08",
               "LANDSAT_8_2018-11-12", "LANDSAT_8_2018-11-17", "LANDSAT_8_2018-11-19", "LANDSAT_8_2018-11-24",
               "LANDSAT_8_2018-11-26", "LANDSAT_8_2018-11-28", "LANDSAT_8_2018-11-30", "LANDSAT_8_2018-12-05",
               "LANDSAT_8_2018-12-19", "LANDSAT_8_2018-12-25", "LANDSAT_8_2018-12-30", "LANDSAT_8_2019-01-01",
               "LANDSAT_8_2019-01-04", "LANDSAT_8_2019-01-06", "LANDSAT_8_2019-01-10", "LANDSAT_8_2019-01-11",
               "LANDSAT_8_2019-01-15", "LANDSAT_8_2019-01-24", "LANDSAT_8_2019-01-26", "LANDSAT_8_2019-01-27",
               "LANDSAT_8_2019-11-04", "LANDSAT_8_2019-11-06", "LANDSAT_8_2019-11-08", "LANDSAT_8_2019-11-10",
               "LANDSAT_8_2019-11-11", "LANDSAT_8_2019-11-15", "LANDSAT_8_2019-11-17", "LANDSAT_8_2019-11-24",
               "LANDSAT_8_2019-11-26", "LANDSAT_8_2019-11-27", "LANDSAT_8_2019-12-03", "LANDSAT_8_2019-12-10",
               "LANDSAT_8_2019-12-12", "LANDSAT_8_2019-12-17", "LANDSAT_8_2019-12-24", "LANDSAT_8_2019-12-26",
               "LANDSAT_8_2019-12-29", "LANDSAT_8_2019-12-31", "LANDSAT_8_2020-01-02", "LANDSAT_8_2020-01-11",
               "LANDSAT_8_2020-01-20", "LANDSAT_8_2020-10-30", "LANDSAT_8_2020-11-08", "LANDSAT_8_2020-11-15",
               "LANDSAT_8_2020-11-17", "LANDSAT_8_2020-11-19", "LANDSAT_8_2020-11-24", "LANDSAT_8_2020-11-26",
               "LANDSAT_8_2020-11-28", "LANDSAT_8_2020-11-29", "LANDSAT_8_2020-12-01", "LANDSAT_8_2020-12-03",
               "LANDSAT_8_2020-12-08", "LANDSAT_8_2020-12-10", "LANDSAT_8_2020-12-14", "LANDSAT_8_2020-12-15",
               "LANDSAT_8_2020-12-21", "LANDSAT_8_2020-12-24", "LANDSAT_8_2020-12-26", "LANDSAT_8_2020-12-30",
               "LANDSAT_8_2021-01-06", "LANDSAT_8_2021-01-13", "LANDSAT_8_2021-01-16", "LANDSAT_8_2021-01-18",
               "LANDSAT_8_2021-01-22", "LANDSAT_8_2021-02-01", "LANDSAT_8_2021-10-31", "LANDSAT_8_2021-11-04",
               "LANDSAT_8_2021-11-09", "LANDSAT_8_2021-11-11", "LANDSAT_8_2021-11-15", "LANDSAT_8_2021-11-20",
               "LANDSAT_8_2021-11-29", "LANDSAT_8_2021-12-01", "LANDSAT_8_2021-12-02", "LANDSAT_8_2021-12-04",
               "LANDSAT_8_2021-12-06", "LANDSAT_8_2021-12-08", "LANDSAT_8_2021-12-11", "LANDSAT_8_2021-12-18",
               "LANDSAT_8_2021-12-20", "LANDSAT_8_2021-12-22", "LANDSAT_8_2021-12-24", "LANDSAT_8_2022-01-05",
               "LANDSAT_8_2022-01-07", "LANDSAT_8_2022-01-12", "LANDSAT_8_2022-01-14", "LANDSAT_8_2022-01-16",
               "LANDSAT_8_2022-01-19", "LANDSAT_8_2022-01-21", "LANDSAT_8_2022-01-25", "LANDSAT_8_2022-01-28",
               "LANDSAT_8_2022-11-07", "LANDSAT_8_2022-11-14", "LANDSAT_8_2022-11-16", "LANDSAT_8_2022-11-18",
               "LANDSAT_8_2022-11-19", "LANDSAT_8_2022-11-21", "LANDSAT_8_2022-11-23", "LANDSAT_8_2022-11-28",
               "LANDSAT_8_2022-12-04", "LANDSAT_8_2022-12-05", "LANDSAT_8_2022-12-11", "LANDSAT_8_2022-12-16",
               "LANDSAT_8_2022-12-18", "LANDSAT_8_2023-01-01", "LANDSAT_8_2023-01-03", "LANDSAT_8_2023-01-10",
               "LANDSAT_8_2023-01-15", "LANDSAT_8_2023-01-22", "LANDSAT_8_2023-01-24", "LANDSAT_8_2023-11-17",
               "LANDSAT_8_2023-11-19", "LANDSAT_8_2023-11-21", "LANDSAT_8_2023-12-05", "LANDSAT_8_2023-12-07",
              # "LANDSAT_8_2023-12-10", # this image is messed up on the GEE side
               "LANDSAT_8_2023-12-19", "LANDSAT_8_2023-12-28", "LANDSAT_8_2023-12-30",
               "LANDSAT_8_2024-01-04", #"LANDSAT_8_2024-01-11", # this image is messed up on the GEE side
               "LANDSAT_8_2024-01-13", "LANDSAT_8_2024-01-22",
               "LANDSAT_8_2024-01-24", "LANDSAT_8_2024-11-03", "LANDSAT_8_2024-11-07", "LANDSAT_8_2024-11-12",
               "LANDSAT_8_2024-11-14", "LANDSAT_8_2024-11-21", #"LANDSAT_8_2024-11-26", # same with this one. Image is there, but it doesn't render under certain zooms on Geemao. 
               "LANDSAT_8_2024-12-09",
               "LANDSAT_8_2024-12-21"])

# Define start and end date for the image collection filter
start_date = "2016-03-06"
end_date = "2025-01-01"

# Define ROI for analysis
roi_for_cropping = roi_LF_crop

In [20]:
def addImageDate(image):
    mission = image.get('SPACECRAFT_ID')
    date = image.date().format('YYYY-MM-dd')
    missDate = ee.String(mission).cat('_').cat(ee.String(date))
    return image.set('missDate', missDate)

def mosaic_by_date(imcol):
    # Convert the image collection to a list of images
    imlist = imcol.toList(imcol.size())
    
    # Get unique dates from the image collection
    def get_date(image):
        return ee.Image(image).date().format("YYYY-MM-dd")
    
    unique_dates = imlist.map(lambda im: get_date(im)).distinct()

    def create_mosaic(date_str):
        date = ee.Date(date_str)
        
        # Filter images for that day and create a mosaic
        mosaic = imcol.filterDate(date, date.advance(1, 'day')).mosaic()
        
        return mosaic.set({
            'system:time_start': date.millis(),
            'system:id': date.format('YYYY-MM-dd')
        })

    # Create mosaics for each unique date
    mosaic_imlist = unique_dates.map(create_mosaic)
    
    return ee.ImageCollection(mosaic_imlist)



# filter image collection
s2 = ee.ImageCollection('LANDSAT/LC08/C02/T2_TOA')\
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])\
    .filterDate(start_date, end_date)\
    .map(addImageDate)\
    .filter(ee.Filter.inList("missDate", ids))\
    .filter(ee.Filter.gt('SUN_ELEVATION',20))\
    .filterBounds(roi_for_cropping)\
    .sort('DATE_ACQUIRED')
s3 = mosaic_by_date(s2)

# Clip all the images in the s3 collection down to the ROI
def clip_image(image):
    return image.clip(roi_for_cropping)

# Apply clip to image collection
l8_clipped = s3.map(clip_image)

In [27]:
# Function to calculate mean band values around a simple point
# this function will be passed through the define endmembers 
def calculate_mean_band_values(image, point_geometry):
    buffer = point_geometry.buffer(30)  # 3x3 Landsat pixel window
    band_means = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=buffer,
        scale=30,
        maxPixels=1e6  # Adjusted based on sample size
    )
    return band_means


# Function to define endmembers
def define_endmembers(image, roi_ice, roi_soil, n=1):
    # Calculate brightness
    brightness = image.select(['B2', 'B3', 'B4']).reduce(ee.Reducer.sum()).rename('brightness') # Red, Green, Blue

    # Generate a grid of points
    points_ice = ee.FeatureCollection.randomPoints(region=roi_ice, points=2000)

    # Calculate brightness for each point
    points_with_brightness_ice = brightness.reduceRegions(
        collection=points_ice,
        reducer=ee.Reducer.mean(),
        scale=30
        )

    # Sort points by brightness
    brightest_point = points_with_brightness_ice.sort('brightness', False).first()
    brightest_geometry = ee.Feature(brightest_point).geometry()

    # Filter out points near the brightest point
    #filtered_points = distinct_points.filterBounds(brightest_geometry.buffer(100))
    #filtered_points_ice = points_with_brightness_ice.filter(ee.Filter.bounds(brightest_geometry.buffer(100)).Not())

    # Find the dimmest point from the remaining points
    points_soil = ee.FeatureCollection.randomPoints(region = roi_soil, points = 2000)

    points_with_brightness_soil = brightness.reduceRegions(
        collection=points_soil,
        reducer=ee.Reducer.mean(),
        scale=30
    )

    dimmest_point = points_with_brightness_soil.sort('brightness', True).first()
    #dimmest_point = filtered_points.sort('brightness', True).first()
    dimmest_geometry = ee.Feature(dimmest_point).geometry()

    # Calculate mean band values for brightest and dimmest points
    brightest_band_means = calculate_mean_band_values(image, brightest_geometry)
    dimmest_band_means = calculate_mean_band_values(image, dimmest_geometry)

    # Combine into endmembers
    endmembers = ee.List([
     brightest_band_means.values(),
     dimmest_band_means.values()
    ])
    return endmembers

In [28]:
def unmix_image_collection(image_collection, roi_ice, roi_soil, n=1):
    
    def process_image(image):
        # Define endmembers for the current image
        endmembers = define_endmembers(image, roi_ice, roi_soil, n=1)

        # Apply the unmixing process
        unmixed = image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']).unmix(endmembers, True, True)

        # Rename unmixed bands for clarity
        unmixed = unmixed.rename(['ice_endmember', 'soil_endmember'])

        # Add properties for tracking
        return unmixed.set('system:time_start', image.get('system:time_start'))

    # Map the unmixing process over the image collection
    unmixed_collection = image_collection.map(process_image)

    return unmixed_collection


In [29]:
# apply process to all images in collection
output_collection = unmix_image_collection(image_collection = l8_clipped, roi_ice = roi_ice_LF, roi_soil = roi_soil_LF)
#result_collection = l8_clipped.map(unmix_image_collection(roi_for_brightness))
print(output_collection.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'ice_endmember', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [2, 2], 'origin': [162, -79], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'soil_endmember', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [2, 2], 'origin': [162, -79], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:time_start': 1478044800000, 'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'ice_endmember', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [2, 2], 'origin': [162, -79], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'soil_endmember', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [2, 2], 'origin': [162, -79], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:time_start': 1478217600000, 'system:index': '1'}}

In [32]:
#### Dropdown to view all the outputs
collection = output_collection.sort('system:time_start')

# Efficiently fetch the list of date strings from system:time_start
image_dates = collection.aggregate_array('system:time_start').getInfo()
date_strs = [ee.Date(d).format('YYYY-MM-dd').getInfo() for d in image_dates]

# Create a folium map centered on an approximate location
center = [-77.60326829880877, 163.26832963921493]

# Output widget for displaying the map
output = Output()

def update_map(change):
    output.clear_output(wait=True)  # Ensure previous content is removed before updating
    with output:
        m = folium.Map(location=center, zoom_start=12)
        date_millis = image_dates[date_strs.index(change.new)]
        image = collection.filter(ee.Filter.eq('system:time_start', date_millis)).first()
        
        if image:
            map_id_dict = image.getMapId()
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                overlay=True,
                name=f"Image {change.new}"
            ).add_to(m)
            
            folium.LayerControl().add_to(m)
        
        display(m)  # Move display() here to ensure rendering

# Create dropdown menu
dropdown = Dropdown(
    options=date_strs,
    description='Date:',
    value=date_strs[0] if date_strs else None
)

dropdown.observe(update_map, names='value')

# Display dropdown and map
ui = VBox([dropdown, output])
ui


In [31]:
#### Dropdown to view all the outputs
collection = l8_clipped.sort('system:time_start')

# Efficiently fetch the list of date strings from system:time_start
image_dates = collection.aggregate_array('system:time_start').getInfo()
date_strs = [ee.Date(d).format('YYYY-MM-dd').getInfo() for d in image_dates]

# Create a folium map centered on an approximate location
center = [-77.60326829880877, 163.26832963921493]

# Output widget for displaying the map
output = Output()

def update_map(change):
    output.clear_output(wait=True)  # Ensure previous content is removed before updating
    with output:
        m = folium.Map(location=center, zoom_start=12)
        date_millis = image_dates[date_strs.index(change.new)]
        image = collection.filter(ee.Filter.eq('system:time_start', date_millis)).first()
        
        if image:
            map_id_dict = image.getMapId()
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                overlay=True,
                name=f"Image {change.new}"
            ).add_to(m)
            
            folium.LayerControl().add_to(m)
        
        display(m)  # Move display() here to ensure rendering

# Create dropdown menu
dropdown = Dropdown(
    options=date_strs,
    description='Date:',
    value=date_strs[0] if date_strs else None
)

dropdown.observe(update_map, names='value')

# Display dropdown and map
ui = VBox([dropdown, output])
ui


In [33]:
# export the images in the unmixed collection
def export_image(image):
    date_str = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'unmixed_images_{date_str}',
        folder='EarthEngine',  # Replace with your folder name
        scale=30,                 # Define the scale (e.g., 30 meters per pixel)
        crs='EPSG:3031',          # Define the Coordinate Reference System
        region=pointed,  # Define the region to export
        fileNamePrefix=f'LANDSAT_FRY_unmix_mar25_{date_str}'
    )
    task.start()
    print(f"Export started for image with date {date_str}.")


    # need to make sure the unmixed collection is called here, not the l8_clipped variable. 
def process_images():
    image_list = output_collection.toList(output_collection.size().getInfo())
    num_images = image_list.size().getInfo()
    
    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        export_image(image)

process_images()

Export started for image with date 2016-11-02.
Export started for image with date 2016-11-04.
Export started for image with date 2016-11-06.
Export started for image with date 2016-11-08.
Export started for image with date 2016-11-13.
Export started for image with date 2016-11-15.
Export started for image with date 2016-12-08.
Export started for image with date 2016-12-10.
Export started for image with date 2016-12-13.
Export started for image with date 2016-12-15.
Export started for image with date 2016-12-17.
Export started for image with date 2016-12-19.
Export started for image with date 2016-12-24.
Export started for image with date 2017-01-02.
Export started for image with date 2017-01-09.
Export started for image with date 2017-01-11.
Export started for image with date 2017-01-14.
Export started for image with date 2017-01-18.
Export started for image with date 2017-01-25.
Export started for image with date 2017-01-27.
Export started for image with date 2017-01-30.
Export starte

In [10]:
# Finding endmember values
# Function to calculate mean band values with buffering
def calc_band_values(image, point_geometry):
    buffer = point_geometry.buffer(60)  # 3x3 Landsat pixel window
    return image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=buffer,
        scale=30,
        maxPixels=1e6
    )

# Function to define endmembers
def find_endmembers(image, roi, n=1):
    # Calculate brightness
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6']).reduce(ee.Reducer.sum()).rename('brightness')

    # Generate a grid of points with a fixed seed for reproducibility
    points = ee.FeatureCollection.randomPoints(region=roi, points=2000)

    # Calculate brightness for each point
    points_with_brightness = brightness.reduceRegions(
        collection=points,
        reducer=ee.Reducer.mean(),
        scale=30
    )

    # Sort points by brightness
    brightest_point = points_with_brightness.sort('brightness', False).first()
    brightest_geometry = ee.Feature(brightest_point).geometry()

    # Filter out points near the brightest point
    filtered_points = points_with_brightness.filter(ee.Filter.bounds(brightest_geometry.buffer(100)).Not())

    # Find the dimmest point from the remaining points
    dimmest_point = filtered_points.sort('brightness', True).first()
    dimmest_geometry = dimmest_point.geometry()

    # Calculate mean band values for brightest and dimmest points
    brightest_band_means = calc_band_values(image, brightest_geometry)
    dimmest_band_means = calc_band_values(image, dimmest_geometry)

    # Combine geometries and band means into a feature
    feature = ee.Feature(None, {
        'image_id': image.id(),
        'date': image.date(),
        'brightest_geometry': brightest_geometry.coordinates(),
        'dimmest_geometry': dimmest_geometry.coordinates(),
        'brightest_band_means': brightest_band_means.values(),
        'dimmest_band_means': dimmest_band_means.values()
    })

    return feature

# Function to map over the image collection
def process_the_collection(collection, roi):
    def map_the_function(image):
        return find_endmembers(image, roi)

    return collection.map(map_the_function)

# Example usage
roi = roi_LF
image = l8_clipped.first()
processed_collection = process_the_collection(l8_clipped, roi)


# Export to CSV
export_task = ee.batch.Export.table.toDrive(
    collection=processed_collection,
    description='Endmembers_Export',
    folder='EarthEngine',
    fileNamePrefix='endmembers_output_LF_20250325',
    fileFormat='CSV'
)

export_task.start()